In [1]:
import Utils
Utils.set_seed(Utils.seed)

import Dataloader,  Model
import torch
import tqdm
import numpy                as  np
import matplotlib.pyplot    as  plt
import torch.nn             as  nn


e:\Thesis\ISAV2023V2\ISAV_2023_V2\Dataloader\__init__.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtered = df_filt[mask2]


# Continuous Wavelet transform (CWT)

In [5]:
import pandas as pd
df = pd.read_csv('Dataset\Gear.csv')

_datas = 20
X_numpy = df['Data' ].values.reshape(_datas,len(df['Data'])//_datas)
Y_numpy = df['Faulty'].values.reshape(_datas,len(df['Data'])//_datas)

In [ ]:
# scales = np.arange(1, Utils.Scales)
# import pywt
# for wavelet in reversed(Utils.wavelet_lis):
#     # Perform Continuous Wavelet Transform
#     print(wavelet)
#     coefficients, frequencies = pywt.cwt(Dataloader.X_numpy[1,:Utils.input_horizon], scales, wavelet)

#     # # Plot the results
#     # plt.figure(figsize=(10, 6))
#     # plt.imshow(np.abs(coefficients), extent=[0, 1, 1, Utils.Scales], aspect='auto', cmap='jet')
#     # plt.colorbar(label='Magnitude')
#     # plt.title('Continuous Wavelet Transform')
#     # plt.xlabel('Time')
#     # plt.ylabel('Scale')
#     # plt.show()

# if Utils.Coefficient_Real:
#     coefficients = np.abs(coefficients)

In [6]:
from    torchvision.models          import  resnet50, ResNet50_Weights
from    torchvision.transforms      import  Resize
import                         pywt


class ResNetFeatureExtractor(nn.Module):
    def __init__(self,
                 cut    = -1,
                 ):
        super(ResNetFeatureExtractor, self).__init__()
        self.resnet  = resnet50(weights = ResNet50_Weights.IMAGENET1K_V2)
        # Remove the classification layer (the last fully connected layer)
        # and pooling layer
        self.resnet = torch.nn.Sequential(*list(self.resnet.children())[:cut])
        self.ssshape = 2048
        self.fc = torch.nn.Sequential(nn.Linear(self.ssshape,500),
                                        # nn.Dropout(p=0.5),
                                        nn.ReLU(),
                                        nn.Linear(500,20),
                                        # nn.Dropout(p=0.5),
                                        nn.ReLU(),
                                        nn.Linear(20,2),)

        self.scales        =     np.arange(1, Utils.Scales)
        
        # for name, param in self.resnet.named_parameters():
        #     if name.startswith('0') or name.startswith('1') or name.startswith('4') or name.startswith('5'):
        #         param.requires_grad = False
        # for param in self.resnet.parameters():
        #         param.requires_grad = False
                
    def _CWT(self,  x:np.array):
        coefficients, _     =    pywt.cwt(x, self.scales, Utils.wavelet)
        if Utils.Coefficient_Real:
            coefficients    =    np.abs(coefficients)

        return coefficients


    def forward(self,
                data,
                ):
        coefficients    =    self._CWT(data)

        coefficients = np.repeat(coefficients[:, :, np.newaxis], 3, axis=2)
        coefficients = torch.tensor(coefficients, dtype=torch.float32).permute(-1,0,1).unsqueeze(0).to(device = Utils.Device)
        
        out = self.resnet(coefficients)
        out = self.fc(out.mean(dim=-1).view(self.ssshape))

        return out.view(1,2)
    def check(self,):
        # Verify requires_grad for each parameter
        for name, param in self.resnet.named_parameters():
            print(name, param.requires_grad)

In [7]:
class Simp_Conv(nn.Module):
    def __init__(self,
                 ):
        super(Simp_Conv, self).__init__()
        self.in_channels = 1
        self.Conv = torch.nn.Sequential(
                                        nn.BatchNorm2d(self.in_channels),
                                        nn.Conv2d(self.in_channels, 50, kernel_size=3),
                                        nn.ReLU(),
                                        nn.Conv2d(50, 50, kernel_size=5,padding = 0,stride=2),
                                        nn.ReLU(),
                                        nn.Conv2d(50, 100, kernel_size=5,padding = 0,stride=2),
                                        nn.ReLU(),
                                        nn.Conv2d(100, 100, kernel_size=5,padding = 0,stride=2),
                                        nn.ReLU(),)

        self.fc = torch.nn.Sequential(
                                        nn.Linear(100,50),
                                        nn.Dropout(p=0.5),
                                        nn.ReLU(),
                                        nn.Linear(50,2),
                                        )

        self.scales        =     np.arange(1, Utils.Scales)
        
    def _CWT(self,  x:np.array):
        coefficients, _     =    pywt.cwt(x, self.scales, Utils.wavelet)
        if Utils.Coefficient_Real:
            coefficients    =    np.abs(coefficients)

        return coefficients


    def forward(self,
                data,
                ):
        coefficients    =    self._CWT(data)

        coefficients = torch.tensor(coefficients, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device = Utils.Device)
        
        out = self.Conv(coefficients)
        # out = self.fc(out.mean(dim=-1).view(self.ssshape))
        out = out.mean(dim=-1).mean(dim=-1)

        out = self.fc(out)
        return out#.view(1,2)
    


In [8]:
prediction_input_size   = Utils.input_horizon
epochs                  = Utils.num_epochs
# Simp_Conv().to(device = Utils.Device)#ResNetFeatureExtractor().to(device = Utils.Device)#
model     = Model.Structure().to(device = Utils.Device)
criterion = torch.nn.CrossEntropyLoss(label_smoothing = 0.3)
optimizer = torch.optim.Adam(
                            model.parameters(),
                            lr=1e-4,
                            # momentum=0.9,
                            weight_decay=1e-3
                            )

In [9]:
loss_t = []
ll = (Dataloader.X_numpy.shape[1]//prediction_input_size)-1
for epoch in range(1,epochs+1):
    print(ll)
    loop_train = tqdm.tqdm(range(ll),total=ll,desc="train",position=0,leave=True)
    loss_train_list = []

    j = 0
    accu = 0
    for Batch in loop_train:#range(Dataloader.X_numpy.shape[0]):
        for test in range(17):#[np.where(Dataloader.Test_ID_numpy[:,0] == data)[0][0] for data in [1,4,5,6,7,9,11,12,19,24,26,27]]:#28,29
            model.train()
            optimizer.zero_grad()
            x = X_numpy[test,Batch*prediction_input_size:(Batch+1)*prediction_input_size]
            y = Y_numpy[test,Batch*prediction_input_size: Batch*prediction_input_size+1]
            x = (x-x.mean())/x.std()
            prediction = model.forward(x)

            if torch.argmax(prediction).item() == y:
                accu +=1
            j += 1    
            
            if y==0:
                y = np.array([1,0])
            else:
                y = np.array([0,1])

            loss_train = criterion(prediction, torch.tensor(y.reshape(1,2),dtype=torch.float32).to(device=Utils.Device))
            # Back propagation
            loss_train.backward()
            # Update model parameters
            optimizer.step()
            loss_train_list.append(loss_train)
            
            if Batch%10 == 0:
                # print(loss_train)
                loop_train.set_description(f"{epoch=}- Test : {test} {y=}")
                loop_train.set_postfix(
                    loss_batch="{:.7f}".format(torch.tensor(loss_train_list).mean()),
                    Accu ="{:.5f}".format(100*accu/j),refresh=True, )
                    
        loss_t.append(torch.tensor(loss_train_list).mean())

162


epoch=1- Test : 16 y=array([0, 1]): 100%|██████████| 162/162 [01:07<00:00,  2.39it/s, Accu=96.01754, loss_batch=0.4563373]


In [12]:
accu_li =[]
ll = (Dataloader.X_numpy.shape[1]//prediction_input_size)-1

with torch.inference_mode():
    for test in range(16,20):#[np.where(Dataloader.Test_ID_numpy[:,0] == data)[0][0] for data in [22,21,23,20]]:
        loop_train = tqdm.tqdm(range(ll),total=ll,desc="train",position=0,leave=True)
        j = 0
        accu = 0
        for Batch in loop_train:
            model.eval()

            x = X_numpy[test,Batch*prediction_input_size:(Batch+1)*prediction_input_size]
            y = Y_numpy[test,Batch*prediction_input_size: Batch*prediction_input_size+1]
            v = model(x)

            if torch.argmax(v).item() == y:
                accu +=1

            j += 1
            if Batch%10 == 0:
                loop_train.set_description(f"Test : {test} {y=}")
                loop_train.set_postfix(
                    loss_batch="{:.7f}".format(torch.tensor(loss_train_list).mean()),
                    Accu ="{:.5f}".format(100*accu/j),refresh=True, )
            accu_li.append(100*accu/j)
# print(100*accu/j)

train:   0%|          | 0/162 [00:00<?, ?it/s]

Test : 19 y=array([0.]): 100%|██████████| 162/162 [00:02<00:00, 55.45it/s, Accu=100.00000, loss_batch=0.4561607]
